In [1]:
import xgboost as xgb
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

In [2]:
train_set = pd.read_csv('train/train_encoded.csv')
train_set.head(10)

,id,len_keyword,len_location,len_text,len_text_original,diff_len_text,keyword_sum,keyword_min,keyword_max,keyword_mean,...,location_min,location_max,location_mean,location_median,text_sum,text_min,text_max,text_mean,text_median,target
0,1,0.0,0.0,43,69,26,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,2.745943,0.224894,0.474996,0.343243,0.323476,1
1,4,0.0,0.0,36,38,2,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,2.545323,0.211650,0.508967,0.363618,0.340166,1
2,5,0.0,0.0,72,133,61,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,3.925010,0.153758,0.585124,0.327084,0.256037,1
3,6,0.0,0.0,50,65,15,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,2.515355,0.150085,0.574226,0.359336,0.346962,1
4,7,0.0,0.0,52,88,36,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,2.937988,0.247769,0.444024,0.326443,0.287533,1
5,8,0.0,0.0,84,110,26,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,3.472170,0.094812,0.392155,0.267090,0.268381,1
6,10,0.0,0.0,74,95,21,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,3.652263,0.211903,0.421501,0.304355,0.292368,1
7,13,0.0,0.0,21,59,38,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,2.169400,0.288043,0.580492,0.433880,0.410864,1
8,14,0.0,0.0,38,79,41,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,2.425554,0.344773,0.491582,0.404259,0.389401,1
9,15,0.0,0.0,24,52,28,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.966376,0.391599,0.639305,0.491594,0.467736,1


In [3]:
train_set.drop('id', axis=1, inplace=True)

### XGBoost

In [4]:
X, y = train_set.iloc[:,:-1],train_set.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

### Random Search

In [5]:
learning_rate = [0.1, 0.3, 0.01, 0.001]
max_depth = [20, 50, 70, 100]
#subsamples = [0.4, 0.5, 0.6]
colsample_bytree = [0.4, 0.5, 0.6]
n_estimators = [20, 50, 70, 100]
objective = ['reg:logistic']

random_search = {'learning_rate': learning_rate, 'max_depth': max_depth,\
                'colsample_bytree': colsample_bytree, 'n_estimators': n_estimators, 'objective': objective}

In [6]:
xgb_random = RandomizedSearchCV(estimator = xgb.XGBClassifier(),\
                               param_distributions=random_search, n_iter=50, cv=7, random_state=111)

In [7]:
xgb_random.fit(X_train, y_train)

RandomizedSearchCV(cv=7,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=100,...
                                           num_parallel_tree=None,
                                           random_state=None, reg_alpha=None,
                                    

In [8]:
xgb_random.best_params_

{'objective': 'reg:logistic',
 'n_estimators': 70,
 'max_depth': 70,
 'learning_rate': 0.001,
 'colsample_bytree': 0.4}

In [9]:
preds = xgb_random.predict(X_test)
f1 = np.sqrt(f1_score(y_test, preds))
print("F1-Mean Score: %f" % (f1))

F1-Mean Score: 0.746268


In [10]:
xg_reg = xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.4,\
                           learning_rate = 0.001, max_depth = 50, n_estimators = 50)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
f1 = np.sqrt(f1_score(y_test, preds))
print("F1-Mean Score: %f" % (f1))

F1-Mean Score: 0.743381


In [18]:
xg_reg = xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.5,\
                           learning_rate = 0.1, max_depth = 100, n_estimators = 100)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
f1 = np.sqrt(f1_score(y_test, preds))
print("F1-Mean Score: %f" % (f1))

F1-Mean Score: 0.771413


0.786182 con id

In [16]:
xg_reg = xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.5,\
                           learning_rate = 0.01, max_depth = 100, n_estimators = 100)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
f1 = np.sqrt(f1_score(y_test, preds))
print("F1-Mean Score: %f" % (f1))

F1-Mean Score: 0.758492


##### Best params de random forest

In [13]:
xg_reg = xgb.XGBClassifier(objective ='reg:logistic', subsample=0.3, colsample_bytree = 0.6,\
                           learning_rate = 0.1, max_depth = 70, alpha = 10, n_estimators = 100)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
f1 = np.sqrt(f1_score(y_test, preds))
print("F1-Mean Score: %f" % (f1))

F1-Mean Score: 0.743071


##### Parámetros que mejor funcionaron con pruebas a mano

In [14]:
xg_reg = xgb.XGBClassifier(objective ='reg:logistic', subsample=0.3,
                colsample_bytree = 0.6, learning_rate = 0.1,
                max_depth = 70, alpha = 10, n_estimators = 57)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
f1 = np.sqrt(f1_score(y_test, preds))
print("F1-Mean Score: %f" % (f1))

F1-Mean Score: 0.738833


### Test

In [26]:
test_set = pd.read_csv('test/test_encoded.csv')

In [27]:
col_id = test_set['id']
test_set.drop('id', axis=1, inplace=True)

In [28]:
preds = xg_reg.predict(test_set)
preds

array([0, 1, 1, ..., 1, 1, 1])

In [29]:
test_set['target'] = preds
test_set['id'] = col_id
test_set[['id', 'target']]

,id,target
0,0,0
1,2,1
2,3,1
3,9,0
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


### Resultado

In [30]:
test_set[['id', 'target']].to_csv('result_xgb.csv', header=True, index=False)